import form csv

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import zipfile
import glob
from scipy.fft import fft, fftfreq

# Settings for file structure
base_path = r"E:\EarthScienceFair_Data"

# Total mass (kg)
mass = 1.8559
results = []

# Find all xlsx files
xlsx_files = glob.glob(os.path.join(base_path, "**", "*.xlsx"), recursive=True)
for tracker_file in xlsx_files:
    print("Deal with:", tracker_file)
    try:
        df = pd.read_excel(tracker_file)
    except:
        print("Failed to read:", tracker_file)
        continue

    # Analyzing
    # 時間軸不一致等問題放在分析部分處理，針對每個要得出的數值去計算需要調整的部分就好
    
    # Above
    t_a = df.iloc[:, -3].values
    x = df.iloc[:, -2].values
    y = df.iloc[:, -1].values

    # Side
    t_s =  df.iloc[:, 0].values
    ye = df.iloc[:, 1].values
    yd = df.iloc[:, 2].values
    yc = df.iloc[:, 3].values
    yb = df.iloc[:, 4].values
    ya = df.iloc[:, 5].values
   
    # To avarage
    t_a = t_a[pd.to_numeric(t_a, errors='coerce').notna()]
    t_a = t_a.astype(float)
    t_a = t_a - np.mean(t_a)
    x = x[pd.to_numeric(x, errors='coerce').notna()]
    x = x.astype(float)
    x = x - np.mean(x)
    y = y[pd.to_numeric(y, errors='coerce').notna()]
    y = y.astype(float)
    y = y - np.mean(y)
    t_s = t_s[pd.to_numeric(t_s, errors='coerce').notna()]
    t_s = t_s.astype(float)
    t_s = t_s - np.mean(t_s)
    ye = ye[pd.to_numeric(ye, errors='coerce').notna()]
    ye = ye.astype(float)  
    ye = ye - np.mean(ye)
    yd = yd[pd.to_numeric(yd, errors='coerce').notna()]
    yd = yd.astype(float)
    yd = yd - np.mean(yd)
    yc = yc[pd.to_numeric(yc, errors='coerce').notna()]
    yc = yc.astype(float)
    yc = yc - np.mean(yc)
    yb = yb[pd.to_numeric(yb, errors='coerce').notna()]
    yb = yb.astype(float)
    yb = yb - np.mean(yb)
    ya = ya[pd.to_numeric(ya, errors='coerce').notna()]
    ya = ya.astype(float)
    ya = ya - np.mean(ya)

    # RMS
    rms_x = np.sqrt(np.mean(x**2))
    rms_y = np.sqrt(np.mean(y**2))
    ratio = rms_y / rms_x if rms_x != 0 else 0
    
    # FFT
    dt = np.mean(np.diff(t_s))
    for y in [ye, yd, yc, yb, ya]:
        N = len(y)
        yf = fft(y)
        xf = fftfreq(N, dt)
        # Positive frequencies only
        xf = xf[:N//2]
        amplitude = 2.0/N * np.abs(yf[0:N//2])
        # Find main frequency
        peak_index = np.argmax(amplitude)
        main_freq = xf[peak_index]
        print("{} main frequency: {:.2f} Hz".format(key, main_freq))

    # Cut 0 Hz (Cut Average)
    peak_index = np.argmax(amplitude[1:]) + 1
    f_n = xf[peak_index]
    
    # 算剛性
    omega = 2 * np.pi * f_n
    k = mass * omega**2
    
    # ==== 加速度檔 ====
    try:
        acc_df = pd.read_csv(acc_file, sep=None, engine='python')
        acc_abs = acc_df.iloc[:, -1].values
        rms_acc = np.sqrt(np.mean(acc_abs**2))
    except:
        rms_acc = np.nan
    
    results.append([tracker_file, rms_x, rms_y, ratio, f_n, k, rms_acc])

# ====== 輸出結果 ======
results_df = pd.DataFrame(results, columns=[
    "File", "RMS_x", "RMS_y", "偏心比例", "主頻率(Hz)", "等效剛性(N/m)", "RMS加速度"
])

print("\n===== 分析結果 =====")
print(results_df)

# 存成總表
results_df.to_csv("analysis_results.csv", index=False)


Deal with: E:\EarthScienceFair_Data\1\G2\1-G1-1.xlsx


AttributeError: 'numpy.ndarray' object has no attribute 'notna'